In [25]:
import pandas as pd
import numpy as np
from numpy.random import default_rng

# Introduction
This notebook explores the classic percolation model for bushfire modelling and considers the Poisson distribution later.

In [52]:
x < .5

array([[ True,  True, False, False,  True],
       [ True, False,  True, False, False],
       [ True, False, False,  True,  True],
       [False, False,  True, False,  True],
       [ True,  True,  True,  True,  True]])

In [83]:
x

array([[0.22733602, 0.31675834, 0.79736546, 0.67625467, 0.39110955],
       [0.33281393, 0.59830875, 0.18673419, 0.67275604, 0.94180287],
       [0.24824571, 0.94888115, 0.66723745, 0.09589794, 0.44183967],
       [0.88647992, 0.6974535 , 0.32647286, 0.73392816, 0.22013496],
       [0.08159457, 0.1598956 , 0.34010018, 0.46519315, 0.26642103]])

In [84]:
index = (1, 1)
x[index]

0.5983087535871898

In [77]:
I, J = np.where(x[:1] < .5)
Z = zip(I, J)

In [60]:
def create_lattice(p, N, mode):
    rng = default_rng(12345)
    if mode == 'bernoulli':
        x = rng.uniform(low=0, high=1, size=(N, N))
        y = x < p
    elif mode == 'poisson':
        y = rng.poisson(lam=p, size=(N, N))
    return y.astype(int)

# returns false if index is outside of lattice
def in_lattice(index, lattice):
    m, n = lattice.shape
    i, j = index
    if not 0 <= i <= m-1 or not 0 <= j <= n-1:
        return False
    else:
        return True

def get_neighbours(node, lattice):
    # get neighbours based on reach, i.e. node value
    # same for bernoulli and poisson modes, reach = L_inf distance
    nbs = []
    i, j = node
    reach = lattice[node]
    
    

In [79]:
def simulate(p, N, mode):
    matrices = []
    lattice = create_lattice(p, N, mode)
    front = []
    archive = []
    
    # add top row to front
    I, J = np.where(lattice[:1] > 0)
    for index in zip(I, J):
        front.append(index)
        
    # spread fire
    While front not empty:
        new_front = []
        mat = np.copy(lattice)
        matrices.append(mat)
        
        for node in front:
            nbs = get_neighbours(node, lattice)
    return matrices

In [80]:
simulate(.5, 5, 'bernoulli')

[(0, 0), (0, 1), (0, 4)]


[]